In [1]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/chicken-disease-classification'

## Entities 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path 
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_learning_rate: float

## configurtions manager in src/config

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        
        create_directories([self.config.prepare_base_model.root_dir])
        return PrepareBaseModelConfig(
            root_dir = Path(self.config.prepare_base_model.root_dir),
            base_model_path = Path(self.config.prepare_base_model.base_model_path),
            updated_base_model_path= Path(self.config.prepare_base_model.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,   
            params_classes = self.params.CLASSES,
            params_learning_rate = self.params.LEARNING_RATE
        )
            

## Components

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf # to call VGG16 from keras

2023-09-14 13:14:20.746423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 13:14:23.938123: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-14 13:14:23.939642: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 13:14:28.382484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
        )

        self.save_model(path= self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes,  freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif(freeze_all is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax'
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs= model.input,
            outputs= prediction
        )

        full_model.summary()

        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.models.Model):
        model.save(path)



## Pipeline

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


Configuration Manager Initiated
[ 2023-09-14 13:14:34,350: INFO: common:  yaml file: config/config.yaml loaded successfully]
[ 2023-09-14 13:14:34,359: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-14 13:14:34,362: INFO: common:  directory: artifacts created successfully]
[ 2023-09-14 13:14:34,364: INFO: common:  directory: artifacts created successfully]
[ 2023-09-14 13:14:34,365: INFO: common:  directory: artifacts/prepare_base_model created successfully]
[ 2023-09-14 13:14:34,366: INFO: common:  directory: artifacts/prepare_base_model created successfully]
[ 2023-09-14 13:14:35,411: WARNING: saving_utils:  Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0     

/workspaces/chicken-disease-classification/env/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
